In [1]:
import cv2
import time
import math
import numpy as np

import mediapipe as mp

from pynput.keyboard import Key, Controller

keyboard = Controller()

wcam, hcam = 1280, 720

cap = cv2.VideoCapture(0)
cap.set(3, wcam)
cap.set(4, hcam)

ptime = 0

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

last_angle = None
last_length = None

min_angle = -95  # min value of volume in the lap
max_angle = 0  # max value of volume in the lap
angle = 0
angle_bar = 400  # initial value 400 to make the bar empty
angle_deg = 0
min_hand = 50
max_hand = 300

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    landmark_list = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmark_list.append([cx, cy])

    if len(landmark_list) != 0:
        x1, y1 = landmark_list[4]
        x2, y2 = landmark_list[8]

        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        cv2.circle(img, (x1, y1), 15, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (0, 0, 255), cv2.FILLED)

        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

        length = math.hypot(x2 - x1, y2 - y1)  # "الوتر" way to determine the distance

        # hand range length 50-300
        angle = np.interp(length, [min_hand, max_hand], [min_angle, max_angle])
        angle_bar = np.interp(length, [min_hand, max_hand], [400, 150])
        angle_deg = np.interp(length, [min_hand, max_hand], [0, 100])

        if last_length:
            if length > last_length:
                keyboard.press(Key.media_volume_up)
                keyboard.release(Key.media_volume_up)
                print("VOL UP")
            elif length < last_length:
                keyboard.press(Key.media_volume_down)
                keyboard.release(Key.media_volume_down)
                print("VOL DOWN")
        last_length = length
        last_angle = angle

        print(int(length), angle)
        if length < 50:  # make the circle with green color when volume is zero
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
    cv2.rectangle(img, (50, 150), (84, 400), (255, 0, 0), 3)  # make bar for the volume

    cv2.rectangle(img, (50, int(angle_bar)), (85, 400), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, f'{int(angle_deg)}%', (40, 120), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 3)

    cv2.imshow('img', img)
    if cv2.waitKey(1) == 27:
        cap.release()
        cv2.destroyAllWindows()
        break


143 -59.55911462880126
VOL UP
155 -55.02160056427438
VOL UP
161 -52.45172951252002
VOL UP
169 -49.60821791563771
VOL UP
175 -47.43488901834535
VOL UP
179 -45.82732218843094
VOL UP
181 -45.00304934274269
VOL UP
184 -43.720543542227084
VOL UP
185 -43.46929746557177
VOL UP
187 -42.93288805642936
VOL UP
188 -42.3964637744755
VOL DOWN
186 -43.10682684489288
VOL UP
194 -40.094151787561486
VOL DOWN
193 -40.38486840329633
VOL UP
195 -39.55878560904584
VOL UP
197 -39.02339031404527
VOL UP
199 -38.24263732151177
VOL DOWN
198 -38.531961732134576
VOL DOWN
197 -38.85890338835878
VOL UP
197 -38.82047885228319
VOL UP
198 -38.487966521884736
VOL DOWN
193 -40.38486840329633
VOL UP
194 -39.931186049728055
VOL DOWN
193 -40.38486840329633
VOL DOWN
193 -40.580302370549084
VOL DOWN
185 -43.642509993604804
VOL DOWN
178 -46.32051714145564
VOL UP
180 -45.53880515211555
VOL DOWN
153 -55.7594986285317
VOL DOWN
137 -61.73652900925925
VOL DOWN
132 -63.59159990636482
VOL DOWN
73 -86.12757635224378
VOL UP
76 -84.915